In [ ]:
def fill_five_digits(num: int) -> str:
    """Given an integer, return string representation, left-pad with 0s to five digits"""
    return str(num).rjust(5, "0")

assert fill_five_digits(123) == "00123"

In [2]:
from enum import IntEnum

class ParameterMode(IntEnum):
    Position = 0
    Immediate = 1

class Opcode(IntEnum):
    Add = 1
    Multiply = 2
    Input = 3
    Output = 4
    JumpIfTrue = 5
    JumpIfFalse = 6
    LessThan = 7
    Equals = 8

In [3]:
class Instruction:

    def __init__(self, instruction: int):
        five_digit_inst: str = fill_five_digits(instruction)
        self.first_param_mode = int(five_digit_inst[2])
        self.second_param_mode = int(five_digit_inst[1])
        self.third_param_mod = int(five_digit_inst[0])
        self.opcode = int(five_digit_inst[3:])

In [4]:
from typing import List, Tuple

def handle_input_instruction(
    memory: List[int], inst_ptr: int, inst_input: int
) -> Tuple[List[int], int]:
    """Handles an input instruction and returns updated memory and instruction pointer"""
    inst: Instruction = Instruction(memory[inst_ptr])
    if inst.opcode != Opcode.Input:
        raise ValueError

    store_addr: int = memory[inst_ptr+1]
    memory[store_addr] = inst_input
    inst_ptr += 2
    return memory, inst_ptr


def handle_output_instruction(memory: List[int], inst_ptr: int) -> Tuple[List[int], int]:
    """Handles an output instruction and returns updated memory and instruction pointer"""
    inst: Instruction = Instruction(memory[inst_ptr])
    if inst.opcode != Opcode.Output:
        raise ValueError

    load_address = memory[inst_ptr+1]
    inst_ptr += 2
    return memory, inst_ptr, memory[load_address]

In [5]:
def handle_add_multiply_instruction(memory: List[int], inst_ptr: int) -> Tuple[List[int], int]:
    """Handles an add/mult instruction and returns updated memory and instruction pointer"""
    inst: Instruction = Instruction(memory[inst_ptr])
    if inst.opcode not in [Opcode.Add, Opcode.Multiply]:
        raise ValueError

    first_param = memory[inst_ptr+1]
    second_param = memory[inst_ptr+2]
    store_addr = memory[inst_ptr+3]

    # If instructions are position, load value from memory.
    # If instructions are immediate, do nothing because param
    # already has immediate value stored.
    if inst.first_param_mode == ParameterMode.Position:
        first_param = memory[first_param]
    if inst.second_param_mode == ParameterMode.Position:
        second_param = memory[second_param]

    if inst.opcode == Opcode.Add:
        memory[store_addr] = first_param + second_param
    elif inst.opcode == Opcode.Multiply:
        memory[store_addr] = first_param * second_param

    inst_ptr += 4

    return memory, inst_ptr

In [6]:
def handle_jump_instruction(memory: List[int], inst_ptr: int) -> Tuple[List[int], int]:
    """Handles jump-if-true and jump-if-false instructions
    
    Returns updated memory and instruction pointer
    """
    inst: Instruction = Instruction(memory[inst_ptr])
    if inst.opcode not in [Opcode.JumpIfTrue, Opcode.JumpIfFalse]:
        raise ValueError

    first_param = memory[inst_ptr+1]
    second_param = memory[inst_ptr+2]

    if inst.first_param_mode == ParameterMode.Position:
        first_param = memory[first_param]

    if inst.second_param_mode == ParameterMode.Position:
        second_param = memory[second_param]

    jump_true_cond = inst.opcode == Opcode.JumpIfTrue and first_param != 0
    jump_false_cond = inst.opcode == Opcode.JumpIfFalse and first_param == 0
        
    if jump_true_cond or jump_false_cond:
        inst_ptr = second_param
    else:
        inst_ptr += 3

    return memory, inst_ptr

In [7]:
def handle_compare_instruction(memory: List[int], inst_ptr: int) -> Tuple[List[int], int]:
    """Handles less-than and equals instructions
    
    Returns updated memory and instruction pointer
    """
    inst: Instruction = Instruction(memory[inst_ptr])
    if inst.opcode not in [Opcode.LessThan, Opcode.Equals]:
        raise ValueError

    first_param = memory[inst_ptr+1]
    second_param = memory[inst_ptr+2]
    store_addr = memory[inst_ptr+3]

    if inst.first_param_mode == ParameterMode.Position:
        first_param = memory[first_param]

    if inst.second_param_mode == ParameterMode.Position:
        second_param = memory[second_param]
    
    less_than_cond = inst.opcode == Opcode.LessThan and first_param < second_param
    equals_cond = inst.opcode == Opcode.Equals and first_param == second_param

    if less_than_cond or equals_cond:
        memory[store_addr] = 1
    else:
        memory[store_addr] = 0

    inst_ptr += 4
    
    return memory, inst_ptr

In [8]:
opcode_to_handler = {
    Opcode.Add: handle_add_multiply_instruction,
    Opcode.Multiply: handle_add_multiply_instruction,
    Opcode.Input: handle_input_instruction,
    Opcode.Output: handle_output_instruction,
    Opcode.JumpIfTrue: handle_jump_instruction,
    Opcode.JumpIfFalse: handle_jump_instruction,
    Opcode.LessThan: handle_compare_instruction,
    Opcode.Equals: handle_compare_instruction,
}

In [9]:
from copy import copy

class Amplifier:

    def __init__(self, phase: int, memory: List[int]):
        self.phase = phase
        self.memory = copy(memory)
        self.inst_ptr = 0
        self._first_input_reached = False

    def run_amplifier(self, amp_input: int) -> int:
        """Runs amplifier through memory and returns output"""

        cur_instruction = Instruction(self.memory[self.inst_ptr])

        while cur_instruction.opcode != 99:

            if cur_instruction.opcode == Opcode.Input:
                if not self._first_input_reached:
                    self.memory, self.inst_ptr = handle_input_instruction(self.memory, self.inst_ptr, self.phase)
                    self._first_input_reached = True
                else:
                    self.memory, self.inst_ptr = handle_input_instruction(self.memory, self.inst_ptr, amp_input)
            elif cur_instruction.opcode == Opcode.Output:
                self.memory, self.inst_ptr, output = handle_output_instruction(self.memory, self.inst_ptr)
                return output
            else:
                inst_handler = opcode_to_handler[cur_instruction.opcode]
                self.memory, self.inst_ptr = inst_handler(self.memory, self.inst_ptr)

            cur_instruction = Instruction(memory[self.inst_ptr])

        return None

In [10]:
with open("./input.txt") as f:
    memory: List[int] = list(map(int, f.read().split(",")))

In [11]:
from itertools import cycle, permutations

max_thruster_input = 0

for phase_settings in permutations(range(5, 10)):
    output = 0

    amplifiers = [
        Amplifier(phase_settings[0], memory),
        Amplifier(phase_settings[1], memory),
        Amplifier(phase_settings[2], memory),
        Amplifier(phase_settings[3], memory),
        Amplifier(phase_settings[4], memory),
    ]

    for i in cycle(range(5)):
        amplifier = amplifiers[i]
        output = amplifier.run_amplifier(output)

        if output is None:
            break

        if i == 4:
            amp_e_output = output

    if amp_e_output > max_thruster_input:
        max_thruster_input = amp_e_output

max_thruster_input

54163586